<a href="https://colab.research.google.com/github/Nilkantg/SIH/blob/main/Words_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

In [179]:
words_corpus = pd.read_csv('/content/words_data.xls')
words_corpus.head()

,Unnamed: 0,Word,0,1,2,3,4,5,6,7,...,54,55,56,57,58,59,60,61,62,63
0,0,A LOT,0.316836,0.439745,7.139941e-08,0.322064,0.408899,-0.012561,0.320306,0.373865,...,0.260467,0.404656,-0.028454,0.251036,0.396338,-0.031925,0.241957,0.388138,-0.034238,0.061003
1,1,A LOT,0.777162,0.594822,1.888133e-07,0.766787,0.569503,-0.029101,0.766939,0.543866,...,0.831578,0.531044,-0.019791,0.839598,0.514735,-0.025115,0.845963,0.498970,-0.028685,0.066329
2,2,A LOT,0.282781,0.426264,9.168880e-08,0.279939,0.391712,0.005966,0.275245,0.354410,...,0.289899,0.329482,-0.034678,0.295946,0.304782,-0.038501,0.301991,0.284427,-0.040737,0.070718
3,3,A LOT,0.613345,0.728356,7.959181e-08,0.623554,0.698023,-0.007444,0.642625,0.672808,...,0.660544,0.768533,-0.024703,0.665910,0.764937,-0.027244,0.669510,0.756921,-0.028706,0.074697
4,4,ABUSE,0.493275,0.434260,8.329591e-08,0.508639,0.422977,-0.002672,0.521094,0.407595,...,0.498473,0.365121,-0.024746,0.509517,0.378327,-0.025282,0.514609,0.389408,-0.024399,0.016817


In [180]:
words_corpus.shape

(984, 66)

In [181]:
grouped_df = words_corpus.groupby('Word').mean().reset_index()
grouped_df.head()

,Word,Unnamed: 0,0,1,2,3,4,5,6,7,...,54,55,56,57,58,59,60,61,62,63
0,A LOT,1.500000,0.497531,0.547297,1.078733e-07,0.498086,0.517034,-0.010785,0.501279,0.486237,...,0.510622,0.508429,-0.026906,0.513122,0.495198,-0.030696,0.514856,0.482114,-0.033091,0.068187
1,ABUSE,6.000000,0.538096,0.599229,1.999922e-07,0.535359,0.579011,-0.007492,0.527097,0.564037,...,0.517238,0.587658,-0.033704,0.516869,0.594988,-0.035804,0.515038,0.601525,-0.036176,0.040946
2,AFRAID,11.500000,0.503858,0.500074,1.612074e-07,0.506212,0.488397,-0.008874,0.506352,0.466763,...,0.496187,0.402481,-0.025625,0.495984,0.384775,-0.029534,0.496297,0.368912,-0.032149,0.099860
3,AGREE,18.000000,0.415888,0.560385,1.489088e-07,0.421631,0.527676,-0.003610,0.431725,0.497645,...,0.450098,0.551487,-0.044162,0.460482,0.551557,-0.045297,0.467044,0.551250,-0.045242,0.060719
4,ALL,23.333333,0.536349,0.671744,1.498440e-07,0.526914,0.659371,-0.004760,0.518129,0.660655,...,0.519347,0.729136,-0.024907,0.513265,0.727319,-0.024632,0.509811,0.722866,-0.023989,0.055413


In [182]:
grouped_df.drop('Unnamed: 0', axis=1, inplace=True)
grouped_df.head()

,Word,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,A LOT,0.497531,0.547297,1.078733e-07,0.498086,0.517034,-0.010785,0.501279,0.486237,-0.018094,...,0.510622,0.508429,-0.026906,0.513122,0.495198,-0.030696,0.514856,0.482114,-0.033091,0.068187
1,ABUSE,0.538096,0.599229,1.999922e-07,0.535359,0.579011,-0.007492,0.527097,0.564037,-0.014100,...,0.517238,0.587658,-0.033704,0.516869,0.594988,-0.035804,0.515038,0.601525,-0.036176,0.040946
2,AFRAID,0.503858,0.500074,1.612074e-07,0.506212,0.488397,-0.008874,0.506352,0.466763,-0.014300,...,0.496187,0.402481,-0.025625,0.495984,0.384775,-0.029534,0.496297,0.368912,-0.032149,0.099860
3,AGREE,0.415888,0.560385,1.489088e-07,0.421631,0.527676,-0.003610,0.431725,0.497645,-0.010413,...,0.450098,0.551487,-0.044162,0.460482,0.551557,-0.045297,0.467044,0.551250,-0.045242,0.060719
4,ALL,0.536349,0.671744,1.498440e-07,0.526914,0.659371,-0.004760,0.518129,0.660655,-0.010242,...,0.519347,0.729136,-0.024907,0.513265,0.727319,-0.024632,0.509811,0.722866,-0.023989,0.055413


In [183]:
grouped_df.shape

(114, 65)

In [184]:
# words_corpus.info()

In [185]:
words_corpus = words_corpus.rename(columns={"Unnamed: 0": "index"})

In [186]:
float_columns = words_corpus.select_dtypes(include='float').columns

words_corpus['mean_value'] = words_corpus[float_columns].mean(axis=1)

words_corpus.head()

,index,Word,0,1,2,3,4,5,6,7,...,55,56,57,58,59,60,61,62,63,mean_value
0,0,A LOT,0.316836,0.439745,7.139941e-08,0.322064,0.408899,-0.012561,0.320306,0.373865,...,0.404656,-0.028454,0.251036,0.396338,-0.031925,0.241957,0.388138,-0.034238,0.061003,0.203231
1,1,A LOT,0.777162,0.594822,1.888133e-07,0.766787,0.569503,-0.029101,0.766939,0.543866,...,0.531044,-0.019791,0.839598,0.514735,-0.025115,0.845963,0.498970,-0.028685,0.066329,0.428796
2,2,A LOT,0.282781,0.426264,9.168880e-08,0.279939,0.391712,0.005966,0.275245,0.354410,...,0.329482,-0.034678,0.295946,0.304782,-0.038501,0.301991,0.284427,-0.040737,0.070718,0.185909
3,3,A LOT,0.613345,0.728356,7.959181e-08,0.623554,0.698023,-0.007444,0.642625,0.672808,...,0.768533,-0.024703,0.665910,0.764937,-0.027244,0.669510,0.756921,-0.028706,0.074697,0.451509
4,4,ABUSE,0.493275,0.434260,8.329591e-08,0.508639,0.422977,-0.002672,0.521094,0.407595,...,0.365121,-0.024746,0.509517,0.378327,-0.025282,0.514609,0.389408,-0.024399,0.016817,0.284694


In [187]:
inputs = words_corpus.drop(['Word', 'mean_value', "index"], axis=1)
targets = words_corpus['Word']

In [188]:
std_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

In [189]:
scaled_inputs = std_scaler.fit_transform(inputs)

In [190]:
scaled_inputs[1]

array([ 2.44606189e+00,  1.08848924e-01,  6.02757425e-01,  2.49912876e+00,
        9.78739725e-02, -2.39234731e+00,  2.61808256e+00,  6.60628086e-02,
       -2.31288079e+00,  2.70008751e+00, -1.33567212e-03, -2.11449215e+00,
        2.71950443e+00, -1.00923971e-01, -1.93707566e+00,  2.83876909e+00,
        1.92416210e-01, -2.11979103e+00,  3.05603773e+00,  1.07736536e-01,
       -1.80970740e+00,  3.12565585e+00,  1.44796417e-02, -1.68341203e+00,
        3.12807030e+00, -6.57003320e-02, -1.60201311e+00,  2.84345903e+00,
        1.38044240e-01, -1.35961115e+00,  3.09259804e+00,  6.02504019e-02,
       -1.35962319e+00,  3.20750598e+00, -6.44674878e-02, -1.45228328e+00,
        3.24375589e+00, -1.57916378e-01, -1.50246650e+00,  2.81844306e+00,
        5.86612889e-02, -3.82553883e-01,  3.06108343e+00, -1.67753637e-02,
       -5.44899601e-01,  3.16419995e+00, -1.22748655e-01, -8.09522641e-01,
        3.20456531e+00, -2.02310714e-01, -9.53363418e-01,  2.78189003e+00,
       -4.20448703e-02,  

In [191]:
encoder = LabelEncoder()
target = encoder.fit_transform(targets)

In [192]:
type(target)

numpy.ndarray

In [193]:
inputs_train, inputs_test, target_train, target_test = train_test_split(scaled_inputs, target, test_size=0.2, random_state=369)

In [194]:
inputs_train.shape

(787, 64)

In [195]:
def np_data(data):
  labels = np.array(data)

  return labels

In [196]:
## Convertint pandas data to numpy
numpy_train_inputs = np_data(inputs_train)
numpy_train_target = np_data(target_train)
numpy_test_inputs = np_data(inputs_test)
numpy_test_target = np_data(target_test)

In [197]:
# Train Dataset
train_dataset = TensorDataset(torch.from_numpy(numpy_train_inputs), torch.from_numpy(numpy_train_target))
# Test Dataset
test_dataset = TensorDataset(torch.from_numpy(numpy_test_inputs), torch.from_numpy(numpy_test_target))

In [198]:
test_dataset.tensors[0].shape

torch.Size([197, 64])

In [199]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [200]:
# obtaining one batch of training data to check the shape
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample output size: ', sample_y.size())
print('Sample output: \n', sample_y)

Sample input size:  torch.Size([32, 64])
Sample input: 
 tensor([[-3.5831e-01,  9.3553e-01,  9.1575e-02,  ...,  9.3949e-01,
          4.2362e-02,  3.6548e-01],
        [-2.5030e+00, -6.7498e-01,  2.7045e-01,  ..., -6.2829e-01,
         -1.4028e+00,  6.2726e-01],
        [-2.7531e-01, -4.0026e-01,  4.5897e-01,  ..., -4.5197e-01,
          4.5029e-02, -5.6797e-01],
        ...,
        [-1.2654e+00, -7.5821e-01,  3.6704e-01,  ..., -5.0373e-01,
          1.5966e-01, -6.7989e-01],
        [-1.6440e+00, -1.1438e+00, -9.2444e-01,  ..., -9.6820e-01,
         -4.6305e-02,  9.6743e-01],
        [-5.7244e-01, -1.1508e-03, -7.4164e-03,  ..., -4.7416e-01,
          2.6427e-01, -2.0387e-01]], dtype=torch.float64)
Sample output size:  torch.Size([32])
Sample output: 
 tensor([ 48, 113, 112,  62,  59, 113,  73,  49,  50,  82,  32, 109,  61,  40,
         20,  25,  14, 113, 100, 113,  17,  60, 109,  91,  65,  56, 113,  47,
         96,  97,  68, 109])


**Building a Model**

In [201]:
# ?nn.LSTM

In [202]:
def create_model(nunits, nlayers):
  class ANN_Model(nn.Module):
    def __init__(self, nunits, nlayers):
      super(ANN_Model, self).__init__()

      self.nunits = nunits
      self.nlayers = nlayers

      self.layers = nn.ModuleDict()
      self.layers["input_layer"] = nn.Linear(64, self.nunits)

      for i in range(self.nlayers):
        self.layers[f"hidden_layer{i+1}"] = nn.Linear(self.nunits, self.nunits)

      self.layers["Output_layer"] = nn.Linear(self.nunits, 114)

      self.softmax = nn.Softmax(dim=1)

    def forward(self, inputs):

      x = F.relu(self.layers["input_layer"](inputs))

      for i in range(self.nlayers):
        x = self.layers[f"hidden_layer{i+1}"](x)

      output = self.layers["Output_layer"](x)

      return output

  #Model
  model = ANN_Model(nunits, nlayers)
  # loss function
  loss_fun = nn.CrossEntropyLoss()
  # optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  return model, loss_fun, optimizer


In [203]:
nunits = 225
nlayer = 5

model, loss_fun, optimizer = create_model(nunits, nlayer)

In [204]:
# # Check the first few batches in train_loader
# for i, (X, Y) in enumerate(train_loader):
#     print(f"Batch {i + 1}")
#     print("X shape:", X.shape)  # Should be [batch_size, num_features]
#     print("Y shape:", Y.shape)  # Should be [batch_size]

#     # Optional: Display the first few samples and labels in the batch
#     print("X samples:", X[:5])
#     print("Y samples:", Y[:5])

#     # Break after the first few batches to avoid too much output
#     if i == 2:
#         break


In [205]:
# Iterate through the train_loader
for data in train_loader:
    X, Y = data  # Unpack the features and labels

    # Display shapes
    print("X shape:", X.shape)  # Expected: [32, 64]
    print("Y shape:", Y.shape)  # Expected: [32]

    # Convert to float for model input
    X = X.float()

    # Forward pass: get model output (yhat)
    yhat = model(X)
    print("yhat shape:", yhat.shape)  # Expected: [32, 114]

    # Calculate the loss (CrossEntropyLoss automatically applies softmax)
    loss = loss_fun(yhat, Y)
    print("Loss:", loss.item())  # .item() gets the scalar value of the loss

    # Backpropagation
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()  # Compute new gradients
    optimizer.step()  # Update model parameters

    break  # To check only the first batch for now


X shape: torch.Size([32, 64])
Y shape: torch.Size([32])
yhat shape: torch.Size([32, 114])
Loss: 4.748127460479736


In [206]:
def train_model(model, loss_fun, optimizer, train_loader, test_loader, epochs):

    nepochs = epochs
    train_acc = []
    train_loss = []

    for epoch in range(nepochs):
        model.train()

        batch_acc = []
        batch_loss = []

        # Training loop
        for inputs, labels in train_loader:
            inputs = inputs.float()

            yhat = model(inputs)

            loss = loss_fun(yhat, labels)

            optimizer.zero_grad()  # Fix the typo: should be zero_grad()
            loss.backward()
            optimizer.step()

            batch_loss.append(loss.item())

            matches = torch.argmax(yhat, axis=1) == labels
            matchesNumeric = matches.float()
            accuracyPct = 100 * torch.mean(matchesNumeric)
            batch_acc.append(accuracyPct)

        # Calculate and store average training accuracy and loss for the epoch
        train_loss.append(np.mean(batch_loss))
        train_acc.append(np.mean(batch_acc))

        # Print epoch statistics (optional)
        if (epoch + 1) % 5 == 0:
          print(f"Epoch {epoch+1}/{nepochs} - "
                f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.2f}% - ")
    return model, train_loss, train_acc


In [207]:
model, training_loss, training_accuracy = train_model(model, loss_fun, optimizer, train_loader, test_loader, epochs=500)

Epoch 5/500 - Train Loss: 3.4432, Train Acc: 21.88% - 
Epoch 10/500 - Train Loss: 2.9563, Train Acc: 26.72% - 
Epoch 15/500 - Train Loss: 2.5749, Train Acc: 33.05% - 
Epoch 20/500 - Train Loss: 2.2960, Train Acc: 38.81% - 
Epoch 25/500 - Train Loss: 2.1740, Train Acc: 40.80% - 
Epoch 30/500 - Train Loss: 1.8557, Train Acc: 47.72% - 
Epoch 35/500 - Train Loss: 1.7793, Train Acc: 49.68% - 
Epoch 40/500 - Train Loss: 1.7054, Train Acc: 51.27% - 
Epoch 45/500 - Train Loss: 1.5612, Train Acc: 53.51% - 
Epoch 50/500 - Train Loss: 1.4179, Train Acc: 56.44% - 
Epoch 55/500 - Train Loss: 1.5741, Train Acc: 55.61% - 
Epoch 60/500 - Train Loss: 1.2332, Train Acc: 62.28% - 
Epoch 65/500 - Train Loss: 1.2942, Train Acc: 62.90% - 
Epoch 70/500 - Train Loss: 1.0622, Train Acc: 68.24% - 
Epoch 75/500 - Train Loss: 0.9940, Train Acc: 68.70% - 
Epoch 80/500 - Train Loss: 1.0539, Train Acc: 69.14% - 
Epoch 85/500 - Train Loss: 0.9483, Train Acc: 69.32% - 
Epoch 90/500 - Train Loss: 0.7676, Train Acc: 76.

In [209]:
# test_acc = []
# test_loss = []

# model.eval()
# with torch.no_grad():
#     batch_test_acc = []
#     batch_test_loss = []

#     # Testing loop
#     for inputs, labels in test_loader:
#         inputs = inputs.float()

#         yhat = model(inputs)

#         loss = loss_fun(yhat, labels)

#         batch_test_loss.append(loss.item())

#         matches = torch.argmax(yhat, axis=1) == labels
#         matchesNumeric = matches.float()
#         accuracyPct = 100 * torch.mean(matchesNumeric)
#         batch_test_acc.append(accuracyPct)

#     # Calculate and store average test accuracy and loss for the epoch
#     test_loss.append(np.mean(batch_test_loss))
#     test_acc.append(np.mean(batch_test_acc))